In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import math
import heapq as heap
import time

In [3]:
df = pd.read_csv('Dataset.csv')

In [4]:
df.drop_duplicates(subset=['SourceAirport', 'DestinationAirport', 'SourceAirport_City', 'SourceAirport_Country',
                               'SourceAirport_Latitude', 'SourceAirport_Longitude', 'SourceAirport_Altitude',
                               'DestinationAirport_City', 'DestinationAirport_Country', 'DestinationAirport_Latitude',
                               'DestinationAirport_Longitude', 'DestinationAirport_Altitude', 'Distance', 'FlyTime', 'Price'], ignore_index=True, inplace=True)


In [5]:
# Create a new column 'SourceAirport_Location' by combining the latitude, longitude, and altitude into a tuple
df['SourceAirport_Location'] = df.apply(lambda row: (row['SourceAirport_Latitude'], row['SourceAirport_Longitude'], row['SourceAirport_Altitude']), axis=1)

# Drop the individual columns since you now have a single combined column
df.drop(['SourceAirport_Latitude', 'SourceAirport_Longitude', 'SourceAirport_Altitude'], axis=1, inplace=True)


In [6]:
# Create a new column 'DestinationAirport_Location' by combining the latitude, longitude, and altitude into a tuple
df['DestinationAirport_Location'] = df.apply(lambda row: (row['DestinationAirport_Latitude'], row['DestinationAirport_Longitude'], row['DestinationAirport_Altitude']), axis=1)

# Drop the individual columns since you now have a single combined column
df.drop(['DestinationAirport_Latitude', 'DestinationAirport_Longitude', 'DestinationAirport_Altitude'], axis=1, inplace=True)


In [7]:
df

,Airline,SourceAirport,DestinationAirport,SourceAirport_City,SourceAirport_Country,DestinationAirport_City,DestinationAirport_Country,Distance,FlyTime,Price,SourceAirport_Location,DestinationAirport_Location
0,Aerocondor,Sochi International Airport,Kazan International Airport,Sochi,Russia,Kazan,Russia,1506.825604,2.330543,172.662342,"(43.449902, 39.9566, 89)","(55.606201171875, 49.278701782227, 411)"
1,Aerocondor,Astrakhan Airport,Kazan International Airport,Astrakhan,Russia,Kazan,Russia,1040.438320,1.798405,132.955783,"(46.2832984924, 48.0063018799, -65)","(55.606201171875, 49.278701782227, 411)"
2,Aerocondor,Chelyabinsk Balandino Airport,Kazan International Airport,Chelyabinsk,Russia,Kazan,Russia,770.508500,1.906214,78.543730,"(55.305801, 61.5033, 769)","(55.606201171875, 49.278701782227, 411)"
3,Aerocondor,Domodedovo International Airport,Kazan International Airport,Moscow,Russia,Kazan,Russia,715.649350,1.699983,170.633416,"(55.40879821777344, 37.90629959106445, 588)","(55.606201171875, 49.278701782227, 411)"
4,Aerocondor,Belgorod International Airport,Kazan International Airport,Belgorod,Russia,Kazan,Russia,1008.253110,1.573957,183.681933,"(50.643798828125, 36.5900993347168, 735)","(55.606201171875, 49.278701782227, 411)"
...,...,...,...,...,...,...,...,...,...,...,...,...
36515,Iberworld,Lifou Airport,Tiga Airport,Lifou,New Caledonia,Tiga,New Caledonia,68.609280,0.446468,60.448087,"(-20.774799346923828, 167.24000549316406, 92)","(-21.096099853515625, 167.8040008544922, 128)"
36516,Iberworld,Nouméa Magenta Airport,Touho Airport,Noumea,New Caledonia,Touho,New Caledonia,205.972279,0.326138,27.701894,"(-22.25830078125, 166.47300720214844, 10)","(-20.790000915527344, 165.25900268554688, 10)"
36517,Iberworld,Nouméa Magenta Airport,Ouvéa Airport,Noumea,New Caledonia,Ouvea,New Caledonia,180.177492,1.123990,38.639613,"(-22.25830078125, 166.47300720214844, 10)","(-20.640600204467773, 166.572998046875, 23)"
36518,Iberworld,Lifou Airport,Ouvéa Airport,Lifou,New Caledonia,Ouvea,New Caledonia,70.962955,0.941287,37.471086,"(-20.774799346923828, 167.24000549316406, 92)","(-20.640600204467773, 166.572998046875, 23)"


In [8]:
def haversine(lat1, lon1, alt1, lat2, lon2, alt2):
    # Convert latitude and longitude from degrees to radians
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])

    # Earth radius in meters (you can use different values based on your dataset)
    R = 6371000

    # Altitude difference
    alt_diff = abs(alt1 - alt2)

    # Haversine formula
    a = math.sin((lat2 - lat1) / 2) ** 2 + math.cos(lat1) * math.cos(lat2) * math.sin((lon2 - lon1) / 2) ** 2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    # Calculate the distance
    distance = R * c + alt_diff

    # distance is in meteres, so we return the value in kilometers
    return distance / 1000 

In [9]:
class Node:
    def __init__(self, airport, city, country, coordination):
        self.airport = airport
        self.city = city
        self.country = country
        self.coordination = coordination


In [10]:
class Edge:
    def __init__(self, airline, distance, fly_time, price):
        self.airline = airline
        self.distance = distance
        self.fly_time = fly_time
        self.price = price

    def get_score(self, w1 = 1, w2 = 700, w3 = 2):
        return self.distance * w1 + self.fly_time * w2 + self.price * w3


In [11]:
dft = df[['Distance', 'FlyTime', 'Price']]

In [12]:
dft

,Distance,FlyTime,Price
0,1506.825604,2.330543,172.662342
1,1040.438320,1.798405,132.955783
2,770.508500,1.906214,78.543730
3,715.649350,1.699983,170.633416
4,1008.253110,1.573957,183.681933
...,...,...,...
36515,68.609280,0.446468,60.448087
36516,205.972279,0.326138,27.701894
36517,180.177492,1.123990,38.639613
36518,70.962955,0.941287,37.471086


In [13]:
dft.corr()

,Distance,FlyTime,Price
Distance,1.000000,0.992977,0.989027
FlyTime,0.992977,1.000000,0.982073
Price,0.989027,0.982073,1.000000


In [14]:
nodes = {}
graph = {}

In [15]:
for data in df.values:
    airline, src_airport, dest_airport, src_city, src_country, dest_city, dest_country, distance, fly_time, price, src_loc, dest_loc = data
    if src_airport not in nodes:
        nodes[src_airport] = Node(src_airport, src_city, src_country, src_loc)
    
    if dest_airport not in nodes:
        nodes[dest_airport] = Node(dest_airport, dest_city, dest_country, dest_loc)
    
    e = Edge(airline, distance, fly_time, price)
    
    if src_airport not in graph:
        graph[src_airport] = {}
    
    if dest_airport not in graph:
        graph[dest_airport] = {}
        
    graph[src_airport][dest_airport] = e

In [16]:
def get_path_details(parent, start):
    path = []
    curr = start
    distance = 0
    fly_time = 0
    price = 0
    
    while start != -1:
        path.append(curr)

        if curr not in parent:
            return None # Return failure

        next = parent[curr]
        
        if next == -1:
            break
            
        e = graph[next][curr]
        
        distance += e.distance
        fly_time += e.fly_time
        price += e.price
        
        curr = next
        
    path = list(reversed(path))

    return path, distance, fly_time, price

In [17]:
def show_result(path, distance, fly_time, price, ex_time, algorithm):
    result = f"""{algorithm}
Exe time: {ex_time}s
.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-
"""
    for i in range(len(path) - 1):
        edge = graph[path[i]][path[i + 1]]

        result += f"""Flight #{i + 1} ({edge.airline}):
From: {path[i]} - {nodes[path[i]].city}, {nodes[path[i]].country}
To: {path[i + 1]} - {nodes[path[i + 1]].city}, {nodes[path[i + 1]].country}
Duration: {edge.distance}km
Time: {edge.fly_time}km
Price: {edge.price}km
----------------------------
"""
    result += f"""Total Price: ${price}
Total Duration: {distance}km
Total Time: {fly_time}h
"""

    return result

In [18]:
def dijkstra(src, dest):
    s_time = time.time()
    pq = []
    heap.heappush(pq, (0, src))
    
    parent = {}
    dist = {}
    dist[src] = 0
    parent[src] = -1
    
    while pq:
            d, u = heap.heappop(pq)
            for v in graph[u]:
                weight = graph[u][v].get_score()
                if (v not in dist) or (dist[v] > dist[u] + weight):
                    dist[v] = dist[u] + weight
                    heap.heappush(pq, (dist[v], v))
                    parent[v] = u
    
    path_details = get_path_details(parent, dest)
    if path_details is None:
       return None

    path, distance, fly_time, price = path_details

    return (dist[dest], path, distance, fly_time, price, time.time() - s_time)

In [19]:
def heuristic(start, goal):
    node1 = nodes[start]
    node2 = nodes[goal]
    return haversine(node1.coordination[0], node1.coordination[1], node1.coordination[2], node2.coordination[0], node2.coordination[1], node2.coordination[2]) / 50

In [20]:
def a_star(src, dest):
    s_time = time.time()
    pq = []
    heap.heappush(pq, (0, 0, src))
    
    parent = {}
    parent[src] = -1

    dist = {}
    dist[src] = 0

    found = False

    count = 0

    while pq:
        s, d, u = heap.heappop(pq)

        # Goal Test
        if u == dest:
            found = True
            break

        for v in graph[u]:
                e = graph[u][v]
                h = heuristic(v, dest)
                score = d + e.get_score()
                if (v not in dist) or (v in dist and dist[v] >= score):
                    dist[v] = score
                    parent[v] = u
                    heap.heappush(pq, (score + h, score, v))

    if not found:
        return None # Return Failure

    path_details = get_path_details(parent, dest)
    if path_details is None:
       return None

    path, distance, fly_time, price = path_details

    return (dist[dest], path, distance, fly_time, price,  time.time() - s_time)
                

In [21]:
res = dijkstra('Imam Khomeini International Airport', 'Raleigh Durham International Airport')
if res is None:
    print("No path was found.")
else:
    result = show_result(res[1], res[2], res[3], res[4], res[5], "Dijkstra")
    with open('06-UIAI4021-PR1-Q1(Dijkstra).txt', 'w') as file:
        file.write(result)
    print(result)

Dijkstra
Exe time: 0.04022407531738281s
.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-
Flight #1 (Mahan Air):
From: Imam Khomeini International Airport - Tehran, Iran
To: Düsseldorf Airport - Duesseldorf, Germany
Duration: 3921.8990130989446km
Time: 5.450957619833174km
Price: 457.8679576636519km
----------------------------
Flight #2 (Lufthansa):
From: Düsseldorf Airport - Duesseldorf, Germany
To: Newcastle Airport - Newcastle, United Kingdom
Duration: 700.4016904718744km
Time: 0.9686450823247468km
Price: 162.00341803579576km
----------------------------
Flight #3 (Jetstar Airways):
From: Newcastle Airport - Newcastle, United Kingdom
To: Melbourne International Airport - Melbourne, Australia
Duration: 834.6449978671836km
Time: 1.4205257067621924km
Price: 117.1949201700933km
----------------------------
Flight #4 (American Airlines):
From: Melbourne International Airport - Melbourne, Australia
To: Charlotte Douglas International Airport - Charlotte, United States
Duration: 791.2302148

In [22]:
res = a_star('Imam Khomeini International Airport', 'Raleigh Durham International Airport')
if res is None:
    print("No path was found.")
else:
    result = show_result(res[1], res[2], res[3], res[4], res[5], "A*")
    with open('06-UIAI4021-PR1-Q1(A-Star).txt', 'w') as file:
        file.write(result)
    print(result)

A*
Exe time: 0.09209489822387695s
.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-
Flight #1 (Mahan Air):
From: Imam Khomeini International Airport - Tehran, Iran
To: Düsseldorf Airport - Duesseldorf, Germany
Duration: 3921.8990130989446km
Time: 5.450957619833174km
Price: 457.8679576636519km
----------------------------
Flight #2 (Lufthansa):
From: Düsseldorf Airport - Duesseldorf, Germany
To: Newcastle Airport - Newcastle, United Kingdom
Duration: 700.4016904718744km
Time: 0.9686450823247468km
Price: 162.00341803579576km
----------------------------
Flight #3 (Jetstar Airways):
From: Newcastle Airport - Newcastle, United Kingdom
To: Melbourne International Airport - Melbourne, Australia
Duration: 834.6449978671836km
Time: 1.4205257067621924km
Price: 117.1949201700933km
----------------------------
Flight #4 (American Airlines):
From: Melbourne International Airport - Melbourne, Australia
To: Charlotte Douglas International Airport - Charlotte, United States
Duration: 791.2302148525258